This notebook downloads daily state COVID-19 data from [The COVID Tracking Project](https://covidtracking.com/) and analyzes it.

### Set Up

Configures [DataFrame](https://devblogs.microsoft.com/dotnet/an-introduction-to-dataframe/), HttpClient

In [64]:
#r "nuget:Microsoft.Data.Analysis"
using Microsoft.Data.Analysis;
using Microsoft.AspNetCore.Html;
using System.Net.Http;

// DataFrame formatter for Jupyter
Formatter<DataFrame>.Register((df, writer) => 
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent)th(c.Name)));
    var rows = new List<List<IHtmlContent>>();
    var take = 5;
    for (var i = 0; i < Math.Min(take, df.Rows.Count); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(i));
        foreach (var obj in df.Rows[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }
    
    var t = table(
        thead(headers),
        tbody(rows.Select(r => tr(r)))
    );
    
    writer.Write(t);
}, "text/html");


var client = new HttpClient();

### Get Daily Data by State

[Model Documentation](https://covidtracking.com/api#apistatesdaily---states-historical-data)

In [69]:
string state = "NY";

DataFrame data;

// download the CSV and load it into a DataFrame
using (var response = await client.GetAsync("https://covidtracking.com/api/states/daily.csv?state=" + state))
using (var stream = await response.Content.ReadAsStreamAsync())
{
    data = DataFrame.LoadCsv(stream, numberOfRowsToRead: 20);
}

// parse the dateChecked column
var columns = data.Columns;
var dateChecked = columns["dateChecked"];
var newDateChecked = new PrimitiveDataFrameColumn<DateTimeOffset>("dateChecked", dateChecked.Length);
for (var i = 0; i < dateChecked.Length; i++)
{
    newDateChecked[i] = DateTimeOffset.Parse((string)dateChecked[i]);
}
columns["dateChecked"] = newDateChecked;

// remove some deprecated/unused columns
columns.Remove("total");
columns.Remove("hash");
columns.Remove("posNeg");

// show first five rows of loaded data
display(data);

0,20200408,NY,159937,231612,<null>,18279,32869,4925,<null>,<null>,<null>,14590,2020-04-09 20:00:00Z,7067,32869,391549,36,799,200,15775,10621,26396
1,20200408,NY,149316,215837,<null>,18079,32669,4593,<null>,<null>,<null>,14590,2020-04-08 20:00:00Z,6268,32669,365153,36,779,586,14642,10453,25095
2,20200408,NY,138863,201195,<null>,17493,32083,4593,<null>,<null>,<null>,14590,2020-04-07 20:00:00Z,5489,32083,340058,36,731,1880,11073,8174,19247
3,20200406,NY,130689,190122,<null>,16837,30203,4504,<null>,<null>,<null>,13366,2020-04-06 20:00:00Z,4758,30203,320811,36,599,2111,9873,8658,18531
4,20200404,NY,122031,180249,<null>,16479,28092,4376,<null>,<null>,<null>,12187,2020-04-05 20:00:00Z,4159,28092,302280,36,594,1709,10332,8327,18659


### Render Chart

In [70]:
#r "nuget:XPlot.Plotly"
using XPlot.Plotly;

var cases = new Graph.Scatter
{
    name = "Cases",
    x = data.Columns["dateChecked"],
    y = data.Columns["positive"]
};

var deaths = new Graph.Scatter
{
    name = "Deaths",
    x = data.Columns["dateChecked"],
    y = data.Columns["death"]
};

var chart = Chart.Plot(new[] { cases, deaths });
chart.WithTitle("Cases vs Deaths: " + state);
display(chart);